### Setup imports and directories
Configures data paths and loads core libraries used throughout the notebook.


In [43]:
#imports and path 

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = "../data"
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)


## Introduction & Purpose of Notebook 01

- This notebook loads raw data
- Performs EDA
- Merges datasets
- Performs feature engineering
- Produces provider-level dataset for modeling


### Load raw train/test files
Reads all train/test beneficiary, inpatient, outpatient, and label CSVs into DataFrames.


In [44]:
#Load ALL datasets

# TRAIN FILES
train_labels = pd.read_csv("../data/Train_labels.csv")
train_inp = pd.read_csv("../data/Train_Inpatientdata.csv")
train_outp = pd.read_csv("../data/Train_Outpatientdata.csv")
train_benef = pd.read_csv("../data/Train_Beneficiarydata.csv")

# TEST FILES
test_providers = pd.read_csv("../data/Test_labels.csv")
test_inp = pd.read_csv("../data/Test_Inpatientdata.csv")
test_benef = pd.read_csv("../data/Test_Beneficiarydata.csv")
test_outp = pd.read_csv("../data/Test_Outpatientdata.csv")

# Display first few rows (check data loading)
train_labels.head(), train_inp.head(), train_outp.head(), train_benef.head(), test_providers.head(), test_inp.head(), test_benef.head(), test_outp.head()

(   Provider PotentialFraud
 0  PRV51001             No
 1  PRV51003            Yes
 2  PRV51004             No
 3  PRV51005            Yes
 4  PRV51007             No,
       BeneID   ClaimID ClaimStartDt  ClaimEndDt  Provider  \
 0  BENE11001  CLM46614   2009-04-12  2009-04-18  PRV55912   
 1  BENE11001  CLM66048   2009-08-31  2009-09-02  PRV55907   
 2  BENE11001  CLM68358   2009-09-17  2009-09-20  PRV56046   
 3  BENE11011  CLM38412   2009-02-14  2009-02-22  PRV52405   
 4  BENE11014  CLM63689   2009-08-13  2009-08-30  PRV56614   
 
    InscClaimAmtReimbursed AttendingPhysician OperatingPhysician  \
 0                   26000          PHY390922                NaN   
 1                    5000          PHY318495          PHY318495   
 2                    5000          PHY372395                NaN   
 3                    5000          PHY369659          PHY392961   
 4                   10000          PHY379376          PHY398258   
 
   OtherPhysician AdmissionDt  ... ClmDiagnosis

### Data overview
Summary statistics for each key training table to understand dimensions and types before feature work.


In [ ]:
# Inspect shape, info, and descriptive stats for key training tables
datasets = {
    'Train_Inpatientdata': train_inp,
    'Train_Outpatientdata': train_outp,
    'Train_Beneficiarydata': train_benef,
    'Train_labels': train_labels,
}

for name, df in datasets.items():
    print(f"\n>>> {name} shape: {df.shape}")
    df.info()
    display(df.describe(include='all'))


### Quick EDA on raw training data
Checks dataset shapes, missing-value rates, and fraud label distribution.


In [45]:
# Basic EDA: Check for missing values and class distribution and add plots (class distribution, claim amounts, etc.)

for name, df in [
    ("train_benef", train_benef),
    ("train_inp", train_inp),
    ("train_outp", train_outp),
    ("train_labels", train_labels),
]:
    print(f"\n{name}: shape={df.shape}")
    print(df.isnull().mean().sort_values(ascending=False).head(8))

print("\nFraud label distribution:")
print(train_labels["PotentialFraud"].value_counts(normalize=True))


train_benef: shape=(138556, 25)
DOD                                0.989744
BeneID                             0.000000
ChronicCond_Cancer                 0.000000
OPAnnualReimbursementAmt           0.000000
IPAnnualDeductibleAmt              0.000000
IPAnnualReimbursementAmt           0.000000
ChronicCond_stroke                 0.000000
ChronicCond_rheumatoidarthritis    0.000000
dtype: float64

train_inp: shape=(40474, 30)
ClmProcedureCode_6     1.000000
ClmProcedureCode_5     0.999778
ClmProcedureCode_4     0.997134
ClmProcedureCode_3     0.976158
ClmDiagnosisCode_10    0.902975
OtherPhysician         0.884123
ClmProcedureCode_2     0.865247
ClmProcedureCode_1     0.428077
dtype: float64

train_outp: shape=(517737, 27)
ClmProcedureCode_6     1.000000
ClmProcedureCode_5     1.000000
ClmProcedureCode_4     0.999996
ClmProcedureCode_3     0.999992
ClmProcedureCode_2     0.999930
ClmProcedureCode_1     0.999687
ClmDiagnosisCode_10    0.997908
ClmDiagnosisCode_9     0.971341
dtype: floa

### Visual EDA
Quick distributions to spot imbalance and reimbursement patterns before merging.


In [ ]:
# Simple visual EDA on key fields
sns.countplot(x=train_labels['PotentialFraud'])
plt.title('Fraud Distribution')
plt.show()

plt.figure(figsize=(8, 4))
plt.hist(train_inp['InscClaimAmtReimbursed'].dropna(), bins=50)
plt.title('Inpatient Reimbursement Distribution')
plt.xlabel('InscClaimAmtReimbursed')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(8, 4))
plt.hist(train_outp['InscClaimAmtReimbursed'].dropna(), bins=50)
plt.title('Outpatient Reimbursement Distribution')
plt.xlabel('InscClaimAmtReimbursed')
plt.ylabel('Frequency')
plt.show()


### Inpatient feature engineering (train)
Converts dates, computes length of stay, and aggregates inpatient metrics per provider.


In [46]:
# Convert dates and compute length of stay

train_inp["ClaimStartDt"] = pd.to_datetime(train_inp["ClaimStartDt"])
train_inp["ClaimEndDt"]   = pd.to_datetime(train_inp["ClaimEndDt"])
train_inp["LOS"] = (train_inp["ClaimEndDt"] - train_inp["ClaimStartDt"]).dt.days

inp_agg = train_inp.groupby("Provider").agg(
    inp_claims=("ClaimID", "nunique"),
    inp_total_reimbursed=("InscClaimAmtReimbursed", "sum"),
    inp_mean_reimbursed=("InscClaimAmtReimbursed", "mean"),
    inp_max_reimbursed=("InscClaimAmtReimbursed", "max"),
    inp_mean_deductible=("DeductibleAmtPaid", "mean"),
    inp_mean_los=("LOS", "mean"),
    inp_unique_bene=("BeneID", "nunique"),
).reset_index()

inp_agg.head()

,Provider,inp_claims,inp_total_reimbursed,inp_mean_reimbursed,inp_max_reimbursed,inp_mean_deductible,inp_mean_los,inp_unique_bene
0,PRV51001,5,97000,19400.000000,42000,1068.0,5.000000,5
1,PRV51003,62,573000,9241.935484,57000,1068.0,5.161290,53
2,PRV51007,3,19000,6333.333333,10000,1068.0,5.333333,3
3,PRV51008,2,25000,12500.000000,21000,1068.0,4.000000,2
4,PRV51011,1,5000,5000.000000,5000,1068.0,5.000000,1


### Outpatient feature engineering (train)
Aggregates outpatient claim counts, reimbursements, deductibles, and unique beneficiaries per provider.


In [47]:
# Outpatient data aggregation

outp_agg = train_outp.groupby("Provider").agg(
    outp_claims=("ClaimID", "nunique"),
    outp_total_reimbursed=("InscClaimAmtReimbursed", "sum"),
    outp_mean_reimbursed=("InscClaimAmtReimbursed", "mean"),
    outp_mean_deductible=("DeductibleAmtPaid", "mean"),
    outp_unique_bene=("BeneID", "nunique"),
).reset_index()

outp_agg.head()

,Provider,outp_claims,outp_total_reimbursed,outp_mean_reimbursed,outp_mean_deductible,outp_unique_bene
0,PRV51001,20,7640,382.000000,0.000000,19
1,PRV51003,70,32670,466.714286,1.000000,66
2,PRV51004,149,52170,350.134228,2.080537,138
3,PRV51005,1165,280910,241.124464,3.175966,495
4,PRV51007,69,14710,213.188406,0.869565,56


### Beneficiary feature engineering (train)
Joins beneficiary info to inpatient claims, derives ages/chronic condition indicators, and aggregates by provider.


In [48]:
# Feature engineering on beneficiary data

train_benef["DOB"] = pd.to_datetime(train_benef["DOB"], errors="coerce")

# attach beneficiary info to inpatient claims
inp_with_bene = train_inp.merge(train_benef, on="BeneID", how="left")

# approximate age at a reference date (say 2010-01-01)
inp_with_bene["age"] = (
    (pd.Timestamp("2010-01-01") - inp_with_bene["DOB"]).dt.days / 365.25
)

# all chronic condition columns
chronic_cols = [c for c in train_benef.columns if c.startswith("ChronicCond_")]
chronic_cols

# define a helper function to compute percentage

def pct(series, predicate):
    return predicate(series).mean()

benef_agg = inp_with_bene.groupby("Provider").agg(
    avg_age=("age", "mean"),
    pct_male=("Gender", lambda x: (x == 1).mean()),
    pct_female=("Gender", lambda x: (x == 2).mean()),
    pct_renal=("RenalDiseaseIndicator", lambda x: (x == "Y").mean()),
    **{f"mean_{c}": (c, "mean") for c in chronic_cols},
).reset_index()

benef_agg.head()

,Provider,avg_age,pct_male,pct_female,pct_renal,mean_ChronicCond_Alzheimer,mean_ChronicCond_Heartfailure,mean_ChronicCond_KidneyDisease,mean_ChronicCond_Cancer,mean_ChronicCond_ObstrPulmonary,mean_ChronicCond_Depression,mean_ChronicCond_Diabetes,mean_ChronicCond_IschemicHeart,mean_ChronicCond_Osteoporasis,mean_ChronicCond_rheumatoidarthritis,mean_ChronicCond_stroke
0,PRV51001,78.568652,0.400000,0.600000,0.400000,1.600000,1.200000,1.200000,1.800000,1.600000,1.200000,1.200000,1.200000,2.000000,1.400000,1.600000
1,PRV51003,70.894173,0.338710,0.661290,0.274194,1.483871,1.419355,1.370968,1.887097,1.629032,1.596774,1.209677,1.112903,1.790323,1.693548,1.887097
2,PRV51007,78.835501,0.333333,0.666667,0.333333,1.333333,1.000000,1.666667,2.000000,2.000000,1.333333,1.000000,1.000000,2.000000,1.666667,1.333333
3,PRV51008,51.627652,0.500000,0.500000,0.000000,1.500000,2.000000,1.500000,1.500000,1.500000,2.000000,1.500000,1.000000,2.000000,2.000000,2.000000
4,PRV51011,97.837098,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


### Dataset Joining Strategy
- Inpatient and outpatient claims are merged using `Provider` + `ClaimID` to align claim-level details.
- Beneficiary information is merged using `BeneID` to attach member demographics and conditions.
- Provider is used for final aggregation to build provider-level features for modeling.


### Merge training features
Combines inpatient, outpatient, and beneficiary aggregates; joins labels; creates binary fraud target.


In [49]:
# Merge all features and prepare final training dataset

from functools import reduce

train_feature_dfs = [inp_agg, outp_agg, benef_agg]

train_features = reduce(
    lambda left, right: pd.merge(left, right, on="Provider", how="outer"),
    train_feature_dfs
)

train_full = train_features.merge(train_labels, on="Provider", how="inner")

# create binary label
train_full["FraudLabel"] = (train_full["PotentialFraud"] == "Yes").astype(int)

train_full.head(), train_full["FraudLabel"].value_counts(normalize=True)
# Save processed training data with features
train_full.to_csv(os.path.join(PROCESSED_DIR, "train_provider_features.csv"), index=False)

### Feature Engineering Explanation
- Aggregation condenses claim-level records into provider summaries for modeling efficiency.
- Provider-level features capture behavior patterns that help flag fraud risk.
- Metrics include claim counts, reimbursement totals, and unique beneficiaries treated.
- Demographic signals (age, chronic conditions) help contextualize provider patient mix.


### Inpatient feature engineering (test)
Repeats inpatient processing for the test set with LOS computation and provider-level aggregates.


In [50]:
# Repeat feature engineering for test dataset
#Inpatient test

test_inp["ClaimStartDt"] = pd.to_datetime(test_inp["ClaimStartDt"])
test_inp["ClaimEndDt"]   = pd.to_datetime(test_inp["ClaimEndDt"])
test_inp["LOS"] = (test_inp["ClaimEndDt"] - test_inp["ClaimStartDt"]).dt.days

test_inp_agg = test_inp.groupby("Provider").agg(
    inp_claims=("ClaimID", "nunique"),
    inp_total_reimbursed=("InscClaimAmtReimbursed", "sum"),
    inp_mean_reimbursed=("InscClaimAmtReimbursed", "mean"),
    inp_max_reimbursed=("InscClaimAmtReimbursed", "max"),
    inp_mean_deductible=("DeductibleAmtPaid", "mean"),
    inp_mean_los=("LOS", "mean"),
    inp_unique_bene=("BeneID", "nunique"),
).reset_index()

### Outpatient feature engineering (test)
Aggregates outpatient claim metrics per provider for the test data.


In [51]:
#Outpatient test

test_outp_agg = test_outp.groupby("Provider").agg(
    outp_claims=("ClaimID", "nunique"),
    outp_total_reimbursed=("InscClaimAmtReimbursed", "sum"),
    outp_mean_reimbursed=("InscClaimAmtReimbursed", "mean"),
    outp_mean_deductible=("DeductibleAmtPaid", "mean"),
    outp_unique_bene=("BeneID", "nunique"),
).reset_index()

### Beneficiary feature engineering (test)
Adds beneficiary details to test inpatient claims, derives demographics/chronic indicators, and aggregates by provider.


In [52]:
#Beneficiary test

test_benef["DOB"] = pd.to_datetime(test_benef["DOB"], errors="coerce")

test_inp_with_bene = test_inp.merge(test_benef, on="BeneID", how="left")

test_inp_with_bene["age"] = (
    (pd.Timestamp("2010-01-01") - test_inp_with_bene["DOB"]).dt.days / 365.25
)

test_chronic_cols = [c for c in test_benef.columns if c.startswith("ChronicCond_")]

test_benef_agg = test_inp_with_bene.groupby("Provider").agg(
    avg_age=("age", "mean"),
    pct_male=("Gender", lambda x: (x == 1).mean()),
    pct_female=("Gender", lambda x: (x == 2).mean()),
    pct_renal=("RenalDiseaseIndicator", lambda x: (x == "Y").mean()),
    **{f"mean_{c}": (c, "mean") for c in test_chronic_cols},
).reset_index()

### Merge test features and save
Combines all test aggregates and writes provider-level features to processed storage.


### Summary & Export
- Final dataset contains one row per provider.
- Includes aggregated claim features across inpatient, outpatient, and beneficiary data.
- Includes the fraud target label for supervised learning.
- Will be consumed in Notebook 02 for model training.


In [53]:
#Merge test features and save

test_feature_dfs = [test_inp_agg, test_outp_agg, test_benef_agg]

test_full = reduce(
    lambda left, right: pd.merge(left, right, on="Provider", how="outer"),
    test_feature_dfs
)

test_full.to_csv(os.path.join(PROCESSED_DIR, "test_provider_features.csv"), index=False)
test_full.head()

,Provider,inp_claims,inp_total_reimbursed,inp_mean_reimbursed,inp_max_reimbursed,inp_mean_deductible,inp_mean_los,inp_unique_bene,outp_claims,outp_total_reimbursed,...,mean_ChronicCond_Heartfailure,mean_ChronicCond_KidneyDisease,mean_ChronicCond_Cancer,mean_ChronicCond_ObstrPulmonary,mean_ChronicCond_Depression,mean_ChronicCond_Diabetes,mean_ChronicCond_IschemicHeart,mean_ChronicCond_Osteoporasis,mean_ChronicCond_rheumatoidarthritis,mean_ChronicCond_stroke
0,PRV51002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,53790.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRV51006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,30720.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRV51009,2.0,15000.0,7500.000000,11000.0,1068.0,3.000000,2.0,37.0,12230.0,...,1.5,1.500000,2.000000,1.500000,1.5,1.500000,1.500000,1.500000,2.000000,1.500000
3,PRV51010,6.0,56000.0,9333.333333,20000.0,1068.0,3.833333,5.0,32.0,8580.0,...,1.0,1.333333,1.666667,1.333333,1.5,1.166667,1.166667,1.333333,1.333333,1.666667
4,PRV51018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190.0,61620.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
